In [1]:
import os
import pandas as pd
import pickle
import ast
import sys
import re

In [2]:
OUT_DIR="/media/SSD/paper_data/"

In [3]:
def get_loss_acc(input_file):

    with open(input_file, 'rb') as f:
        dest_object_name = pickle.load(f)

    train_loss = dest_object_name[dest_object_name.find("loss"):dest_object_name.find("]")+1]
    train_accuracy = dest_object_name[dest_object_name.find("accuracy"):dest_object_name.find("val_loss")-3]
    validation_loss = dest_object_name[dest_object_name.find("val_loss"):dest_object_name.find("val_accuracy")-3]
    validation_accuracy = dest_object_name[dest_object_name.find("val_accuracy"):dest_object_name.find("lr")-3]

    train_loss_list = ast.literal_eval(train_loss[train_loss.find("["):train_loss.find("]")+1])
    train_accuracy_list = ast.literal_eval(train_accuracy[train_accuracy.find("["):train_accuracy.find("]")+1])
    validation_loss_list = ast.literal_eval(validation_loss[validation_loss.find("["):validation_loss.find("]")+1])
    validation_accuracy_list = ast.literal_eval(validation_accuracy[validation_accuracy.find("["):validation_accuracy.find("]")+1])
    
    return train_accuracy_list, validation_accuracy_list

In [49]:
sensor_tag="all"
seeds = [0]
models = ["LSTM", "CNN", "CNN_small", "LSTM+CNN", "CNN+LSTM", "MLP", "resnet"]
datasets = ["nops/10k", "random/10k", "random/20k"]#, "snippets/10k"]

columns = ['dataset', 'model', 'seed', 'kfold', 'accuracy']

accuracies = pd.DataFrame(columns=columns)

results_idx = 0

for seed in seeds:
    for model in models:
        for dataset in datasets:
            
            dataset_small = dataset.replace("nops/10k", "Exp-OUT-10k-N").replace("random/10k", "Exp-OUT-10k-R").replace("random/20k", "Exp-OUT-20k-R")
        
            if(model == 'resnet'):
                out_path = OUT_DIR+"/alveo/"+dataset+"/results/"+model+"_pnone_s"+str(seed)+"-sensors-"+sensor_tag+"_10fold/out/"
                acc_file=out_path+"kfold_results.txt"

                file1 = open(acc_file, 'r')
                Lines = file1.readlines()

                for count, line in enumerate(Lines):
                    #print("Line{}: {}".format(count, line.strip()))
                    if(count>=1 and count <=10):
                        acc_evol=out_path+"log/LSTM+CNN_kf"+str(count-1)+".txt"
                        kf_accuracy = re.findall("\d+\.\d+", line.strip())[1]
                        #row = pd.DataFrame(['Exp-IN', 'LSTM+CNN', 'none', 0, count, kf_accuracy])
                        #accuracies = pd.concat([accuracies, row], axis=0, ignore_index=True)
                        accuracies.loc[results_idx] = [dataset_small, model, seed, count-1, float(kf_accuracy)]
                        results_idx = results_idx+1
                    if(count==12):
                        kf_accuracy = re.findall("\d+\.\d+", line.strip())[0]
                        accuracies.loc[results_idx] = [dataset_small, model, seed, "avg", float(kf_accuracy)]
                        results_idx = results_idx+1

                file1.close()
            else:
                out_path = OUT_DIR+"/alveo/"+dataset+"/results/"+model+"_pnone_s"+str(seed)+"-sensors-"+sensor_tag+"/out/"
                acc_file=out_path+"/record/LSTM+CNN_record.txt"

                file1 = open(acc_file, 'r')
                Lines = file1.readlines()

                for count, line in enumerate(Lines):
                    #print("Line{}: {}".format(count, line.strip()))
                    if(count==4):
                        accuracy = re.findall("\d+\.\d+", line.strip())[0]
                        #row = pd.DataFrame(['Exp-IN', 'LSTM+CNN', 'none', 0, count, kf_accuracy])
                        #accuracies = pd.concat([accuracies, row], axis=0, ignore_index=True)
                        accuracies.loc[results_idx] = [dataset_small, model, seed, "-", float(accuracy)]
                        results_idx = results_idx+1

                file1.close()

In [55]:
accuracies[(accuracies.kfold=='avg') | (accuracies.kfold=='-')].round(2)

,dataset,model,seed,kfold,accuracy
0,Exp-OUT-10k-N,LSTM,0,-,38.43
1,Exp-OUT-10k-R,LSTM,0,-,29.67
2,Exp-OUT-20k-R,LSTM,0,-,33.01
3,Exp-OUT-10k-N,CNN,0,-,38.74
4,Exp-OUT-10k-R,CNN,0,-,32.69
5,Exp-OUT-20k-R,CNN,0,-,34.40
6,Exp-OUT-10k-N,CNN_small,0,-,2.70
7,Exp-OUT-10k-R,CNN_small,0,-,2.70
8,Exp-OUT-20k-R,CNN_small,0,-,32.40
9,Exp-OUT-10k-N,LSTM+CNN,0,-,40.15


In [48]:
sensor_tag="all"
seeds = [0, 1, 2, 3, 4]
models = ["LSTM", "CNN", "CNN_small", "LSTM+CNN", "CNN+LSTM", "MLP", "resnet"]
datasets = ["snippets/10k"]

columns = ['dataset', 'model', 'seed', 'accuracy']

accuracies_snippets = pd.DataFrame(columns=columns)

results_idx = 0


for model in models:
    for dataset in datasets:
        for seed in seeds:
            
            dataset_small = dataset.replace("snippets/10k", "Exp-OUT-10k-S")
                
            out_path = OUT_DIR+"/alveo/"+dataset+"/results/"+model+"_pnone_s"+str(seed)+"-sensors-"+sensor_tag+"_1fold/out/"
            acc_file=out_path+"/record/LSTM+CNN_record.txt"
            
            file1 = open(acc_file, 'r')
            Lines = file1.readlines()
            
            for count, line in enumerate(Lines):
                #print("Line{}: {}".format(count, line.strip()))
                if(count==4):
                    accuracy = re.findall("\d+\.\d+", line.strip())[0]
                    #row = pd.DataFrame(['Exp-IN', 'LSTM+CNN', 'none', 0, count, kf_accuracy])
                    #accuracies = pd.concat([accuracies, row], axis=0, ignore_index=True)
                    accuracies_snippets.loc[results_idx] = [dataset_small, model, seed, float(accuracy)]
                    results_idx = results_idx+1
            
            file1.close()
        
        accuracies_snippets.loc[results_idx] = [dataset_small, model, 'avg', accuracies_snippets[(accuracies_snippets.model==model) & (accuracies_snippets.dataset==dataset_small)].accuracy.mean()]
        results_idx = results_idx+1

In [56]:
accuracies_snippets#[accuracies_snippets.seed=='avg'].round(2)

,dataset,model,seed,accuracy
0,Exp-OUT-10k-S,LSTM,0,13.680001
1,Exp-OUT-10k-S,LSTM,1,13.706666
2,Exp-OUT-10k-S,LSTM,2,79.799998
3,Exp-OUT-10k-S,LSTM,3,13.440000
4,Exp-OUT-10k-S,LSTM,4,13.453333
5,Exp-OUT-10k-S,LSTM,avg,26.815999
6,Exp-OUT-10k-S,CNN,0,100.000000
7,Exp-OUT-10k-S,CNN,1,100.000000
8,Exp-OUT-10k-S,CNN,2,100.000000
9,Exp-OUT-10k-S,CNN,3,100.000000


In [41]:
# Save output files
accuracies.to_csv(OUT_DIR+"/alveo/accuracies/accuracies_all.csv", index=False)
accuracies[accuracies.kfold=='avg'].to_csv(OUT_DIR+"/alveo/accuracies/accuracies_averaged.csv", index=False)
accuracies_snippets.to_csv(OUT_DIR+"/alveo/accuracies/accuracies_snippets_all.csv", index=False)
accuracies_snippets[accuracies_snippets.seed=='avg'].to_csv(OUT_DIR+"/alveo/accuracies/accuracies_snippets_averaged.csv", index=False)